## Importing the necessary modules

In [1]:
from json import loads
from keras import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, Softmax
from keras.optimizers import Adam
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from numpy import array
from random import choice, shuffle
from string import punctuation
'''
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
'''


'\nnltk.download("punkt")\nnltk.download("wordnet")\nnltk.download(\'omw-1.4\')\n'

## Loading the dataset

In [2]:
data = loads(open("dataset/intents.json").read())


## Using NLP to clean the data

In [3]:
lemmatizer = WordNetLemmatizer()

# Each list to create
words = [] # tokenized words of sentences in patterns
classes = [] # tags
doc_X = [] # patterns
doc_y = [] # tags ocurring number of times wrt patterns

'''Looping through all the intents and tokenizing each patterns
and appending tokens to words, the patterns 
and the associated tag to their associated list'''
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    # add the tag to the classes if it's not there already
    if intent["tag"] not in classes:
        classes.append(intent["tag"])


In [4]:
'''lemmatizing all the words in the vocab 
and converting them to lowercase
if the words don't appear in punctuation'''
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in punctuation] # removes punctuation and converts to lowercase


In [5]:
'''sorting the vocab and classes in alphabetical order 
and taking the set to ensure no duplicates occur'''
# removes duplicates and sorts in alphabetical order
words = sorted(set(words)) 
classes = sorted(set(classes))


In [6]:
print("lenght of words: ", len(words))
print("length of doc_X", len(doc_X))
print("length of doc_y", len(doc_y))
print("length of classes", len(classes))


lenght of words:  115
length of doc_X 111
length of doc_y 111
length of classes 23


## Creating training and test sets

In [7]:
# list for training data
training = []

# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower()) # lemmatizing the sentence and converting to lowercase
    for word in words:
        bow.append(1) if word in text else bow.append(0) # one hot encoding the words if they appear in the text
    output_row = [0] * len(classes)
    # marking the index of class that the current pattern is associated to as 1
    output_row[classes.index(doc_y[idx])] = 1
    # adding the one hot encoded BoW and associated classes to training
    training.append([bow, output_row])


In [8]:
# shuffling the data and convert it to a numpy array
shuffle(training)
training = array(training, dtype=object)


In [9]:
# splitting the features and target labels
train_X = array(list(training[:, 0])) # features
train_y = array(list(training[:, 1])) # target labels


In [10]:
train_X


array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 0],
       ...,
       [0, 0, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [11]:
train_y


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [12]:
print(f"Shape of train_X: {train_X.shape}")
print(f"Shape of train_y: {train_y.shape}")


Shape of train_X: (111, 115)
Shape of train_y: (111, 23)


# Building deep learning model

In [13]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
print("input_shape: ", input_shape)
print("output_shape: ", output_shape)


input_shape:  (115,)
output_shape:  23


In [14]:
# defining the model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation=LeakyReLU(alpha=0.3)))
model.add(Dropout(0.25))
model.add(Dense(64, activation=LeakyReLU(alpha=0.3)))
model.add(Dropout(0.25))
model.add(Dense(output_shape, activation=Softmax()))


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-11-21 17:52:42.451483: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-21 17:52:42.451666: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
# compiling the model
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])


In [16]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               14848     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 23)                1495      
                                                                 
Total params: 24,599
Trainable params: 24,599
Non-trainable params: 0
_________________________________________________________________
None


## Training the model

In [17]:
model.fit(x=train_X, y=train_y, epochs=200)


Epoch 1/200


2022-11-21 17:52:42.772796: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-21 17:52:43.119506: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 32ms/step - loss: 3.1037 - accuracy: 0.1081
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 2.6266 - accuracy: 0.4595
Epoch 3/200
4/4 [==============================] - 0s 11ms/step - loss: 2.0171 - accuracy: 0.6216
Epoch 4/200
4/4 [==============================] - 0s 11ms/step - loss: 1.3678 - accuracy: 0.7387
Epoch 5/200
4/4 [==============================] - 0s 11ms/step - loss: 0.8874 - accuracy: 0.8018
Epoch 6/200
4/4 [==============================] - 0s 11ms/step - loss: 0.5675 - accuracy: 0.8829
Epoch 7/200
4/4 [==============================] - 0s 11ms/step - loss: 0.3337 - accuracy: 0.9459
Epoch 8/200
4/4 [==============================] - 0s 11ms/step - loss: 0.2184 - accuracy: 0.9550
Epoch 9/200
4/4 [==============================] - 0s 11ms/step - loss: 0.2057 - accuracy: 0.9459
Epoch 10/200
4/4 [==============================] - 0s 12ms/step - loss: 0.1230 - accuracy: 0.9820
Epoch 11/200
4/4 [=============

# Functions for getting results

In [18]:
# defining a function to clean the text
# tokenizing and then lemmatizing and returning the tokens array
def clean_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

# defining a function to get the bag of words
# one hot encoding the words if they appear in the text and returning numpy array of bow
def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word == w:
                bow[idx] = 1
    return array(bow)

# defining a function to predict the class
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

# defining a function to get the response 
# taking the predicted class and returning a random response from the intents.json file
def get_response(intents_list, intents_json):
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = choice(i["responses"])
            break
    return result



## Testing the model

In [23]:
from datetime import datetime
import platform
import pyttsx3
now = datetime.now()
engine = pyttsx3.init()
VOICE_ID = "com.apple.speech.synthesis.voice.veena.premium"  # for macOS
if platform.system() == "Windows":  # for Windows
    VOICE_ID = r"HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0"
engine.setProperty('voice', VOICE_ID)
print("Anuradha is ready to chat! (type 'exit' to quit)")
while True:
    try:
        message = input("You : ")
        if message == 'exit':
            TEXT = "Bye! take care"
            print("You :", message)
            print("Anuradha :", TEXT)
            engine.say(TEXT)
            engine.runAndWait()
            break
        intents = pred_class(message, words, classes)
        result = get_response(intents, data)
        if result == "date":
            TEXT = now.strftime(r'%d/%m/%Y')
            print("You :", message)
            print("Anuradha :", TEXT)
            engine.say(TEXT)
            engine.runAndWait()
        elif result == "time":
            TEXT = now.strftime(r'%H:%M:%S')
            print("You :", message)
            print("Anuradha :", TEXT)
            engine.say(TEXT)
            engine.runAndWait()
        else:
            print("You :", message)
            print("Anuradha :", result)
            engine.say(result)
            engine.runAndWait()
    except ValueError:
        TEXT = "Sorry, I didn't get that"
        print("You :", message)
        print("Anuradha :", TEXT)
        engine.say(TEXT)
        engine.runAndWait()


Anuradha is ready to chat! (type 'exit' to quit)
1/1 [==============================] - 0s 24ms/step
You : date
Anuradha : 21/11/2022
1/1 [==============================] - 0s 19ms/step
You : time
Anuradha : 18:22:25
You : exit
Anuradha : Bye! take care
